In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
df = pd.read_excel('emi_summary.xlsx')

df.shape

(153838, 4)

In [3]:
states = pd.DataFrame({'nombre': df['ESTADO'].unique()})

states.head(32)

,nombre
0,Aguascalientes
1,Baja California
2,Baja California Sur
3,Campeche
4,Coahuila De Zaragoza
5,Colima
6,Chiapas
7,Chihuahua
8,Ciudad de Mexico (antes Distrito Federal modif...
9,Durango


In [4]:
engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(
                                os.environ['DBUSER'], 
                                os.environ['DBPASS'], 
                                os.environ['DBHOST'], 
                                os.environ['DBPORT'], 
                                os.environ['DBNAME']))

states.to_sql('estado', con=engine, if_exists='append', index=False)

In [5]:
municipality = df[['ESTADO', 'MUNICIPIO']]
municipality = municipality.drop_duplicates()

municipality.head()

,ESTADO,MUNICIPIO
0,Aguascalientes,Aguascalientes
591,Aguascalientes,Asientos
670,Aguascalientes,Calvillo
825,Aguascalientes,Cosio
846,Aguascalientes,Jesus Maria (Aguascalientes)


In [6]:
municipality.shape

(2456, 2)

In [7]:
municipality.iloc[1000]['ESTADO']

'Oaxaca'

In [8]:
municipality.iloc[2000]['ESTADO']

'Tamaulipas'

In [9]:
municipality.iloc[2100]['ESTADO']

'Veracruz De Ignacio De La Llave'

In [10]:
states_dict = { states.iloc[i]['nombre']: (i+1) for i in range(32)}

states_dict

{'Aguascalientes': 1,
 'Baja California': 2,
 'Baja California Sur': 3,
 'Campeche': 4,
 'Chiapas': 7,
 'Chihuahua': 8,
 'Ciudad de Mexico (antes Distrito Federal modificado 03 de febrero de 2016)': 9,
 'Coahuila De Zaragoza': 5,
 'Colima': 6,
 'Durango': 10,
 'Guanajuato': 11,
 'Guerrero': 12,
 'Hidalgo': 13,
 'Jalisco': 14,
 'Michoacán de Ocampo': 16,
 'Morelos': 17,
 'México': 15,
 'Nayarit': 18,
 'Nuevo Leon': 19,
 'Oaxaca': 20,
 'Puebla': 21,
 'Queretaro': 22,
 'Quintana Roo': 23,
 'San Luis Potosi': 24,
 'Sinaloa': 25,
 'Sonora': 26,
 'Tabasco': 27,
 'Tamaulipas': 28,
 'Tlaxcala': 29,
 'Veracruz De Ignacio De La Llave': 30,
 'Yucatan': 31,
 'Zacatecas': 32}

In [11]:
municipality['ESTADO'] = municipality['ESTADO'].apply(lambda x: states_dict[x])
municipality = municipality.rename(columns={'ESTADO': 'id_estado', 'MUNICIPIO':'nombre'})

municipality.shape

(2456, 2)

In [12]:
municipality.iloc[2100]['id_estado']

30

In [13]:
municipality.to_sql('municipio', con=engine, if_exists='append', index=False)

In [14]:
city = df[['MUNICIPIO', 'CP', 'LOCALIDAD']]

municipality_dict = { municipality.iloc[i]['nombre']: (i+1) for i in range(2456)}

In [15]:
city['MUNICIPIO'] = city['MUNICIPIO'].apply(lambda x: municipality_dict[x])
city = city.rename(columns={'MUNICIPIO':'id_municipio', 'CP': 'cp', 'LOCALIDAD': 'nombre'})

city.shape

/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(153838, 3)

In [ ]:
city.to_sql('localidad', con=engine, if_exists='append', index=False)